# 보스톤 집값 맞추기
1. train, test 데이터 분리
2. 각각 전처리 해주기 - min max scaler. <br>
  train 데이터는 fit_transform(), test데이터는 transform()으로 해주면
  test 데이터는 train에서 학습된 mean, v로 변환 되는 것이니까 이렇게 해줘.
  target data 제외하고 scailing해야함. <br>
  train 데이터를 train validation set으로 분햇
  
3. 모델학습 - randomforest, xgboost, lightgbm 회귀 예측이니까 regressor 3개 써보기
4. 성능확인 - rmse 지표 mean squared error , rmse 낮을수록 좋은 모델
5. test data 예측

In [1]:
from sklearn.datasets import load_boston
import pandas as pd

In [2]:
df = pd.DataFrame(load_boston().data, columns = load_boston().feature_names)
df['target'] = load_boston().target

In [21]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split

In [34]:
#  train, test 데이터 분리
xcol= df.columns.difference(['target'])
x = df[xcol]
y = df['target']
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

In [35]:
# 1. train, test 의 x 값만 min, max scaler로 스케일링
# 2, train 셋을 train, validation으로 분리. 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_xtr = scaler.fit_transform(xtrain)
scaled_xte = scaler.fit_transform(xtest)

In [48]:
# train셋을 train valid 셋으러 분리
from sklearn import linear_model
xtrain2, xvalid, ytrain2, yvalid = train_test_split(scaled_xtr, ytrain, test_size = 0.2, random_state = 42)

# 모델 학습 회위예측 1. xgb, randomforest, lightgbm
import xgboost
import lightgbm

In [51]:
from lightgbm import LGBMRegressor

In [63]:
from xgboost import XGBRegressor

In [64]:
LGBregressor = LGBMRegressor(random_State= 42)
XGBRegressor = XGBRegressor(random_State= 42)

LGBmodel = LGBregressor.fit(xtrain2,ytrain2 )
XGBmodel = XGBRegressor.fit(xtrain2, ytrain2)


[LightGBM] [Warning] Unknown parameter: random_State
[14:52:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "random_State" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [78]:
# SCORE  - RMSE, accuracy
from sklearn.metrics import mean_squared_error
LGBpred = LGBmodel.predict(xvalid)
XGBpred = XGBmodel.predict(xvalid)

LGBrmse =  mean_squared_error(yvalid, LGBpred)**(1/2)
XGBrmse = mean_squared_error(yvalid, XGBpred)**(1/2)
print(LGBrmse)
print(XGBrmse)

print(LGBmodel.score(xvalid, yvalid))
print(XGBmodel.score(xvalid, yvalid))

3.6731852462704695
3.1901150085849754
0.8102347384012002
0.8568656956519156


In [74]:
help(LGBmodel.score)

Help on method score in module sklearn.base:

score(X, y, sample_weight=None) method of lightgbm.sklearn.LGBMRegressor instance
    Return the coefficient of determination :math:`R^2` of the
    prediction.
    
    The coefficient :math:`R^2` is defined as :math:`(1 - \frac{u}{v})`,
    where :math:`u` is the residual sum of squares ``((y_true - y_pred)
    ** 2).sum()`` and :math:`v` is the total sum of squares ``((y_true -
    y_true.mean()) ** 2).sum()``. The best possible score is 1.0 and it
    can be negative (because the model can be arbitrarily worse). A
    constant model that always predicts the expected value of `y`,
    disregarding the input features, would get a :math:`R^2` score of
    0.0.
    
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
        Test samples. For some estimators this may be a precomputed
        kernel matrix or a list of generic objects instead with shape
        ``(n_samples, n_samples_fitted)``, where ``n_sample

In [82]:
# TEST 데이터 예측
testpred = LGBmodel.predict(xtest)
testpred2 = XGBmodel.predict(xtest)
print(mean_squared_error(ytest, testpred)**(1/2))
print(LGBmodel.score(scaled_xte, ytest))
print(XGBmodel.score(scaled_xte, ytest))  # accuracy 48% 정도임

15.632834506226603
0.3890413752255669
0.4866621593348154


In [86]:
mean_squared_error(ytest, testpred2)**(1/2)

15.010169565328768

In [83]:
# submission 파일 만들기
sub  = pd.DataFrame()
sub['index'] = xtest.index
sub['target'] = testpred2

In [85]:
sub.to_csv('submission.csv',index = False)